# 

In [1]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
from matplotlib import colors
from copy import copy


In [2]:
#
# 1D tent function
#
def tent_function(x,i):
    if x<=float(i-1):
        return 0.0
    elif x>float(i+1):
        return 0.0
    elif x<=float(i):
        return x-float(i-1)
    else:
        return float(i+1)-x
#
# Derivative of 1D tent function
#
def dtent_function(x,i):
    if x<=float(i-1):
        return 0.0
    elif x>float(i+1):
        return 0.0
    elif x<=float(i):
        return 1.0
    else:
        return -1.0
#
# Overlap integral of v_i*v_j (assumes i and j are equal or nearest neighbour)
#
def vv_integral(i,j,n):
    if i==j:
        if i==0 or i==n-1:
            return 1.0/3.0
        else:
            return 2.0/3.0
    else:
        return 1.0/6.0
#
# Overlap integral of d_v_i*d_v_j (assumes i and j are equal or nearest neighbour)
#
def dvdv_integral(i,j,n):
    if i==j:
        if i==0 or i==n-1:
            return 1.0
        else:
            return 2.0
    else:
        return -1.0
#
# Overlap integral of d_v_i*v_j (assumes i and j are equal or nearest neighbour)
#
def dvv_integral(i,j,n):
    if i==j:
        if i==0:
            return -1.0/2.0
        elif i==n-1:
            return 1.0/2.0
        else:
            return 0.0
    else:
        if i<j:
            return -1.0/2.0
        else:
            return 1.0/2.0

In [3]:
#
# Define body dimensions in metres and mass density in kg/metre^3 
#
lx=1.0 
ly=0.1
lz=0.01
rho=2710.0 #kg/m3
#
# Define mesh number in x and y directions (must be even)
#
nx=100
ny=20
#
# Define resolution of final displacement fields (for plotting) as a fraction of the mesh sidelengths 
#
plotting_fraction=0.1
#
# Define 2D elasticity (Elastic moduli unit is Pa)
#
bulkModulus=100e9
shearModulus=25e9
bcz=0  #2D plane stress (1 gives 2D plane strain)
#
# Define zero displacement surface boundary conditions
#
bc0=1  #zero displacement on left boundary
bc1=0  #zero traction on lower boundary
bc2=0  #zero disaplacement on right boundary
bc3=0  #zero traction on upper boundary
#
# Scale factor for plotting displacements
#
scaleFactor=10.0

In [4]:
#
# Calculate mesh side lengths
#
lx_mesh=lx/float(nx) 
ly_mesh=ly/float(ny)
#
# Define body force as m*g=a**3*rho*g (g=9.8 metres/second**2)
#
bodyForceMagnitude=lx_mesh*ly_mesh*lz*rho*9.8
#
# Convert elastic moduli to a force per unit mesh
#
bulkModulus=bulkModulus*lx_mesh*ly_mesh
shearModulus=shearModulus*lx_mesh*ly_mesh
#
# Fix internal variables whose values depend on plane strain or plane stress boundary conditions
#
if bcz==0:
    #
    # Plane stress (slides 27-28)
    #
    m_p=2.0*shearModulus*(6.0*bulkModulus+2.0*shearModulus)/(3.0*bulkModulus+shearModulus)
    m_m=2.0*shearModulus*(6.0*bulkModulus-2.0*shearModulus)/(3.0*bulkModulus+shearModulus)
else:
    #
    # Plane strain (slides 25-26)
    #
    m_p=bulkModulus+4.0*shearModulus/3.0 
    m_m=bulkModulus-2.0*shearModulus/3.0

In [5]:
#
# Define constant matrices and vectors
#
columnSize=2*(nx-bc0-bc2)*(ny-bc1-bc3)
F=np.zeros(columnSize) #body forces
M=np.zeros((columnSize,columnSize)) # slide 16 for 1D case
Lambda=np.zeros((columnSize,columnSize)) # slide 16 for 1D case
#
# Construct M and Lambda matrices and F column vector
#
for ix in range(bc0,nx-bc2):
    #
    # Check for x-direction boundary conditions
    #
    if ix==bc0:
        djx_m=0
        djx_p=2
    elif ix==nx-bc2-1:
        djx_m=-1
        djx_p=1
    else:
        djx_m=-1
        djx_p=2
    for iy in range(bc1,ny-bc3):
        #
        # Check for y-direction boundary conditions
        #
        if iy==bc1:
            djy_m=0
            djy_p=2
        elif iy==ny-bc3-1:
            djy_m=-1
            djy_p=1
        else:
            djy_m=-1
            djy_p=2
        alpha=(ix-bc0)+(iy-bc1)*(nx-bc0-bc2)
        #
        # Body force column vector
        #
        F[2*alpha]=0.0
        F[2*alpha+1]=bodyForceMagnitude
        #
        # Matrices
        #
        for djx in range(djx_m,djx_p):
            for djy in range(djy_m,djy_p):
                jx=ix+djx
                jy=iy+djy
                beta=(jx-bc0)+(jy-bc1)*(nx-bc0-bc2)
                #
                # Diagonal M matrix
                #
                M[2*alpha,2*beta]=vv_integral(ix,jx,nx)*vv_integral(iy,jy,ny)
                M[2*alpha+1,2*beta+1]=vv_integral(ix,jx,nx)*vv_integral(iy,jy,ny)
                #
                # Lambda Matrix
                #
                Lambda[2*alpha,2*beta]=m_p*dvdv_integral(ix,jx,nx)*vv_integral(iy,jy,ny)+shearModulus*vv_integral(ix,jx,nx)*dvdv_integral(iy,jy,ny)
                Lambda[2*alpha,2*beta+1]=m_m*dvv_integral(ix,jx,nx)*dvv_integral(jy,iy,ny)+shearModulus*dvv_integral(jx,ix,nx)*dvv_integral(iy,jy,ny)
                Lambda[2*alpha+1,2*beta]=shearModulus*dvv_integral(ix,jx,nx)*dvv_integral(jy,iy,ny)+m_m*dvv_integral(jx,ix,nx)*dvv_integral(iy,jy,ny)    
                Lambda[2*alpha+1,2*beta+1]=m_p*dvdv_integral(iy,jy,ny)*vv_integral(ix,jx,nx)+shearModulus*dvdv_integral(ix,jx,nx)*vv_integral(iy,jy,ny)

In [6]:
#
# Define displacement and velocity vectors
#
U=np.zeros(columnSize) #displacement coefficients
V=np.zeros(columnSize) #displacement velocity coefficients
#
# Choose time step and number of iterations
#
deltaTime=1e-2
itNum=800
#
# Pre-calculate matrices and vectors (coming from Lambda and F)
#
inv_GammaPlus=inv(np.identity(columnSize)+Lambda*deltaTime**2/4.0)
GammaMinus=np.identity(columnSize)-Lambda*deltaTime**2/4.0
M1=np.matmul(inv_GammaPlus,GammaMinus)
M2=np.matmul(inv_GammaPlus,Lambda)*deltaTime
V1=np.matmul(inv_GammaPlus,F)*deltaTime
iLambdaM=np.matmul(inv(M),Lambda)
#
# Integrate equations of motion
#
lastEffectiveForce=np.matmul(iLambdaM,U)+F
for it in range(itNum):
    #
    # Calculate displacement field and store in Ovito dump file
    #
    f=open("evolution."+str(it)+".ssv","w+")
    f.write("%d \r\n" % ((nx-bc2-bc0)*(ny-bc3-bc1)))
    f.write("Properties=pos:R:3:ux:R:1:uy:R:1:umag:R:1:\r\n")
    for ix in range(bc0,nx-bc2):
        for iy in range(bc1,ny-bc3):
            alpha=(ix-bc0)+(iy-bc1)*(nx-bc0-bc2)
            f.write("%f %f %f %f %f %f\r\n" % (ix*lx_mesh+U[2*alpha],iy*ly_mesh+U[2*alpha+1],0.0,U[2*alpha],U[2*alpha+1],np.sqrt(U[2*alpha]**2+U[2*alpha+1]**2)))
    f.close()
    #
    # Update displacement and displacement-velocity fields
    #
    VOld=copy(V)
    UOld=copy(U)

    V=np.matmul(M1,VOld)-np.matmul(M2,UOld)-V1
    U=UOld+(V+VOld)*deltaTime/2.0
    
